In [17]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers



In [18]:
img_height, img_width = 128, 128
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/train",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical'
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/val",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical'
)   
test_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/test",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical'
)

Found 99 files belonging to 6 classes.
Found 20 files belonging to 6 classes.
Found 20 files belonging to 6 classes.


In [19]:
num_classes = len(train_ds.class_names)

In [20]:
normalization_layer = layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))


In [21]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.1),
])
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

In [22]:
def mobile_net_v2_model(num_classes):   
    base_model = keras.applications.MobileNetV2(
        input_shape=(128, 128, 3),
        include_top=False,
        weights='imagenet'
    )

    base_model.trainable = False  # freeze pretrained layers

    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [23]:
mnv2_model = mobile_net_v2_model(num_classes)

history = mnv2_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20
)


Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 765ms/step - accuracy: 0.1313 - loss: 2.3868 - val_accuracy: 0.2000 - val_loss: 1.9469
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 484ms/step - accuracy: 0.3434 - loss: 1.6871 - val_accuracy: 0.3000 - val_loss: 1.7448
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 454ms/step - accuracy: 0.4848 - loss: 1.3451 - val_accuracy: 0.3000 - val_loss: 1.7952
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 439ms/step - accuracy: 0.5556 - loss: 1.1665 - val_accuracy: 0.3500 - val_loss: 1.7324
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 519ms/step - accuracy: 0.6667 - loss: 1.0615 - val_accuracy: 0.4000 - val_loss: 1.7257
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 497ms/step - accuracy: 0.6768 - loss: 0.8894 - val_accuracy: 0.3500 - val_loss: 1.6712
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 425ms/step - accuracy: 0.7273 - loss: 0.8060 - val_accuracy: 0.2000 - val_loss: 1.8572
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 481ms/step - accuracy: 0.7172 - loss: 0.7605 - val_accuracy: 0.3500 - val_loss:

In [24]:
mnv2_model.save('food_classification_model.keras')

In [25]:
scores = mnv2_model.evaluate(test_ds)
print("Test loss:", scores[0])
print("Test accuracy:", scores[1])


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 524ms/step - accuracy: 0.3500 - loss: 2.0866
Test loss: 2.086646556854248
Test accuracy: 0.3499999940395355
